# Firebase

In [90]:
import firebase_admin
from firebase_admin import credentials, firestore
from google.cloud.firestore_v1 import Client
from dotenv import load_dotenv

load_dotenv()

cred = credentials.ApplicationDefault()
firebase_admin.initialize_app()

firestore_client: Client = firestore.client()

ValueError: The default Firebase app already exists. This means you called initialize_app() more than once without providing an app name as the second argument. In most cases you only need to call initialize_app() once. But if you do want to initialize multiple apps, pass a second argument to initialize_app() to give each app a unique name.

In [106]:
tracks_ref = firestore_client.collection(u'tracks_clean_tags')
docs = tracks_ref.stream()

In [107]:
import pandas as pd

df = pd.DataFrame([doc.to_dict() for doc in docs])
df.set_index("id", inplace=True)

In [108]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0bVtnNhxaqhTx56y4kPotO to 7wCmS9TTVUcIhRalDYFgPy
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   duration          50 non-null     int64  
 1   tags              50 non-null     object 
 2   key               50 non-null     int64  
 3   valence           50 non-null     float64
 4   liveness          50 non-null     float64
 5   artist_genres     50 non-null     object 
 6   mode              50 non-null     int64  
 7   danceability      50 non-null     float64
 8   artist_names      50 non-null     object 
 9   name              50 non-null     object 
 10  tempo             50 non-null     float64
 11  pitches           50 non-null     object 
 12  acousticness      50 non-null     float64
 13  energy            50 non-null     float64
 14  loudness          50 non-null     float64
 15  instrumentalness  50 non-null     float64
 16  time_signa

In [109]:
df

,duration,tags,key,valence,liveness,artist_genres,mode,danceability,artist_names,name,tempo,pitches,acousticness,energy,loudness,instrumentalness,time_signature
id,,,,,,,,,,,,,,,,,
0bVtnNhxaqhTx56y4kPotO,232840,"[german, hip-hop, hip hop, rap]",11,0.6830,0.1120,[german pop],0,0.462,[Alligatoah],Fick ihn doch,175.810,"{'120.29057': [0.359, 0.218, 0.152, 0.155, 0.1...",0.180000,0.878,-5.027,0.000000,4
0bYg9bo50gSsH3LtXe2SQn,241106,"[pop, christmas, mariah carey, xmas]",7,0.3500,0.0708,"[urban contemporary, dance pop]",1,0.336,[Mariah Carey],All I Want for Christmas Is You,150.273,"{'157.84349': [0.104, 0.052, 0.082, 0.334, 1.0...",0.164000,0.627,-7.463,0.000000,4
0lLxYP7Z3j8Xum8VGYEkTJ,194861,"[hardcore, comedy, metal, comedy metal, humorc...",6,0.5830,0.3380,"[comic metal, alternative metal, industrial me...",1,0.506,[Psychostick],From the Heart (I Hate You),149.917,"{'175.31166': [0.182, 0.385, 0.218, 0.192, 0.2...",0.644000,0.467,-9.344,0.022600,3
1SOClUWhOi8vHZYMz3GluK,237720,"[alternative, electropop, folk, indie pop, ind...",11,0.4940,0.3040,"[pop, modern rock, indie poptimism]",0,0.671,[Jaymes Young],Infinity,121.963,"{'213.82677': [0.069, 0.058, 0.349, 0.042, 0.0...",0.148000,0.673,-6.941,0.000053,4
1cKHdTo9u0ZymJdPGSh6nq,241066,"[techno, rnb, sad, soul]",10,0.1550,0.1630,"[pop, dark clubbing, destroy techno, canadian ...",1,0.323,"[The Weeknd, Gesaffelstein]",I Was Never There,113.260,"{'156.11401': [0.032, 0.121, 0.081, 0.13, 0.12...",0.142000,0.737,-4.008,0.000162,4
1jDJFeK9x3OZboIAHsY9k2,183440,"[80s, elton john, pop, rock, classic rock]",6,0.7720,0.1400,"[mellow gold, piano rock, glam rock]",1,0.504,[Elton John],I'm Still Standing,176.808,"{'44.30939': [0.966, 0.762, 0.064, 0.091, 0.18...",0.356000,0.904,-6.863,0.121000,4
1yYr4D1KjhhyDD5bZIiFQy,302466,"[metal, metalcore, thrash metal, heavy metal]",9,0.3380,0.3460,"[groove metal, melodic thrash, metalcore, alte...",1,0.477,[Trivium],In Waves,76.517,"{'67.03016': [1.0, 0.647, 0.216, 0.199, 0.083,...",0.000211,0.940,-3.435,0.051800,4
227jOVNY1MrE5MkUcQRhsk,352560,"[metal, symphonic death metal, melodic death m...",1,0.0351,0.5940,"[symphonic deathcore, blackened deathcore]",0,0.290,[Lorna Shore],Pain Remains I: Dancing Like Flames,125.021,"{'32.44295': [1.0, 0.927, 0.266, 0.24, 0.407, ...",0.000166,0.799,-4.361,0.894000,4
27RYrbL6S02LNVhDWVl38b,214333,"[avantgarde pop, post-revolutionary pop song, ...",2,0.4140,0.7180,"[album rock, art rock, experimental, beatlesqu...",1,0.321,"[Yoko Ono, John Lennon]",Happy Xmas (War Is Over) - Remastered 2010,146.172,"{'95.95605': [0.096, 0.091, 0.086, 0.079, 0.29...",0.280000,0.640,-10.023,0.000000,3


In [110]:
tolower = lambda s: s.lower()
flatmap = lambda list_of_lists: [item for l in list_of_lists for item in l]
tags = pd.Series(flatmap(df[~df["tags"].isna()]["tags"].values.tolist()))\
    .apply(tolower)\
    .drop_duplicates()
tags.info()

<class 'pandas.core.series.Series'>
Int64Index: 128 entries, 0 to 186
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
128 non-null    object
dtypes: object(1)
memory usage: 2.0+ KB


In [111]:
tags

0               german
1              hip-hop
2              hip hop
3                  rap
4                  pop
            ...       
181    christmas songs
183          mistagged
184         remastered
185         fight club
186          post-punk
Length: 128, dtype: object